# Bio-inspired Algorithms using Intel® SigOpt

In this section, we will learn how to optimize the parameters and hyperparameters of bio-inspired using the Intel® SigOpt.

## What is the Intel® SigOpt platform?

<p style='text-align: justify;'>
This platform is designed to help researchers, data scientists, and engineers automatically optimize the parameters and hyperparameters of their models, algorithms, and systems, rather than performing the optimization manually.
</p>    

## How does hyperparameter optimization work?

<p style='text-align: justify;'>
Hyperparameter optimization is a common problem in machine learning. Machine learning algorithms, from logistic regression to neural networks, rely on fine-tuned hyperparameters for maximum effectiveness. Different hyperparameter optimization strategies have varying performance and cost (in time, money, and computation cycles).  Evaluating optimization strategies is not intuitive. Stochastic optimization strategies produce a distribution of the most found values. We can generalize this problem as given a function that accepts inputs and returns a numerical output, how can we efficiently find the inputs, or parameters, that maximize the output of the function. Since finding the best hyperparameters of a model is an arduous task that demands a lot of time and resources, how can Intel® SigOpt find the hyperparameters that best fit the proposed model quickly and efficiently.
</p>  

## How to usage?

<p style='text-align: justify;'>
To use the platform, you must first register on the website Intel® SigOpt and then install the library.
</p>    

#### ⊗ Install library

In [1]:
!pip install sigopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 1.0 MB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 14.0 MB/s eta 0:00:00


#### ⊗ Insert the API Key

<p style='text-align: justify;'>To use the platform's API, as a last step, you must enter the key right after registration. Run the code below, click on the link, redeem your key and enter it.</p>

In [ ]:
%load_ext sigopt
%sigopt config

SigOpt API token (find at https://app.sigopt.com/tokens/info):

## ☆ Challenge: Traveling through Spain ☆

Consider the following problem:
<p style="text-align: justify;">
After years dedicated to tending and protecting his precious plantations on the farm, a farmer finally decided to treat himself to well-deserved vacations and set off on a journey through the beautiful landscapes of Spain. He then established the goal of visiting eight renowned cities during this unique adventure.
</p>

<p style="text-align: justify;">
Recognizing the value of time, the farmer understands the importance of finding the most efficient route to travel through all the cities, ensuring a seamless journey with the shortest travel time between each destination.
</p>
    
<p style="text-align: justify;">
The map of Spain takes the form of a graph, where cities are represented as nodes and the roads connecting them as edges. Each edge is associated with a specific distance in miles between each pair of cities. Although the farmer is free to choose a starting city, he must visit exactly eight distinct cities, avoiding revisits during his expedition.
</p>

<p style="text-align: center;">
 <img src="./images/figure08_cities.png"  width="500" height="500">
</p>

<p style="text-align: justify;">
Your mission is to assist the farmer in creating the optimal order to visit all the cities, allowing him to explore their wonders while spending the least amount of time on the road. To do that, answer the following items:
</p>

a) Implement Intel® SigOpt in solutions using **AG**, **PSO** and **ACO**.

b) Compare the obtained **results and hyperparameters**.

c) Measure the **execution time** of this algorithm.

### ☆ Solution ☆ 

<p style='text-align: justify;'>Here are some examples already seen in previous notebooks, but using Intel® SigOpt to perform parameter optimization.</p>

### ⊗ Genetic algorithm

<p style='text-align: justify;'>Below we will perform the optimization of the hyperparameters of the genetic algorithm relative to the solution of the previously proposed problem.</p>

#### Import Intel® SigOpt  and others libraries

<p style='text-align: justify;'>In this step, you can either include the library or use the key provided after registering on the site (We chose to import the library, as it is the most common and practical).</p>

In [2]:
import random
import time
import matplotlib.pyplot as plt
import sigopt

#### Implements genetic algorithm functions

<p style='text-align: justify;'>In this part you just have to create the functions, necessary for the implementation of the genetic algorithm, and provide the problem data.</p>

In [ ]:
# City distance table (indexes are used as city representations)
distance_table = [
    [0, 620, 956, 621, 590, 349, 814, 296],
    [620, 0, 730, 395, 796, 969, 280, 324],
    [956, 730, 0, 335, 368, 609, 528, 660],
    [621, 395, 335, 0, 401, 642, 193, 325],
    [590, 796, 368, 401, 0, 241, 594, 726],
    [349, 969, 609, 642, 241, 0, 835, 645],
    [814, 280, 528, 193, 594, 835, 0, 518],
    [296, 324, 660, 325, 726, 645, 518, 0]
]

# Mapping city indexes to their respective names
city_names = {
    0: "Barcelona",
    1: "Bilbao",
    2: "Jaén",
    3: "Madrid",
    4: "Murcia",
    5: "Valencia",
    6: "Valladolid",
    7: "Zaragoza"
}

# Objective function (total path distance)
def total_distance(path):
    dist = 0
    for i in range(len(path) - 1):
        dist += distance_table[path[i]][path[i + 1]]
    dist += distance_table[path[-1]][path[0]] 
    return dist

# Step 1: Creating the population
def create_population(population_size, num_cities):
    population = []
    for _ in range(population_size):
        path = list(range(num_cities))
        random.shuffle(path)
        population.append(path)
    return population

# Step 2: Evaluation function (calculates fitness for each individual)
def evaluate_population(population):
    return [total_distance(path) for path in population]

# Step 3: Selection of parents (roulette wheel method)
def roulette_selection(population, fitness):
    total_fitness = sum(fitness)
    probabilities = [fit / total_fitness for fit in fitness]
    selected_parents = random.choices(population, weights=probabilities, k=2)
    return selected_parents

# Step 4: Crossover (partial order)
def crossover(parent1, parent2,CROSSOVER_RATE):
    if random.random() < CROSSOVER_RATE:
        start = random.randint(0, len(parent1) - 1)
        end = random.randint(start + 1, len(parent1))
        child = [-1] * len(parent1)

        for i in range(start, end):
            child[i] = parent1[i]

        idx = 0
        for city in parent2:
            if city not in child:
                while child[idx] != -1:
                    idx += 1
                child[idx] = city
        return child
    else:
        return parent1

# Step 5: Mutation (switching positions between two cities)
def mutate(path,MUTATION_RATE):
    if random.random() < MUTATION_RATE:
        idx1, idx2 = random.sample(range(len(path)), 2)
        path[idx1], path[idx2] = path[idx2], path[idx1]

<p style='text-align: justify;'>The function below is the same as previously seen in the genetic algorithm notebook, with only one modification, it must be passed parameters for which we want to optimize.</p>

In [ ]:
# Run the genetic algorithm
def genetic_algorithm(population_size, max_generations, crossover_rate, mutation_rate):
    NUM_CITIES = len(distance_table)
    population = create_population(population_size, NUM_CITIES)
    best_distance = float('inf')
    best_path = []

    for generation in range(max_generations):
        fitness = evaluate_population(population)
        best_idx = fitness.index(min(fitness))
        current_best_path = population[best_idx]
        current_best_distance = total_distance(current_best_path)

        if current_best_distance < best_distance:
            best_distance = current_best_distance
            best_path = current_best_path[:]

        if generation == max_generations - 1:
            print(f"Generation {generation + 1}: Best Distance = {best_distance}")
            print("Best Path:", [city_names[idx] for idx in best_path])
            print("---------------------------------------")
            return best_path

        new_population = []
        for _ in range(population_size // 2):
            parent1, parent2 = roulette_selection(population, fitness)
            child1 = crossover(parent1, parent2,crossover_rate)
            child2 = crossover(parent2, parent1,crossover_rate)
            mutate(child1,mutation_rate)
            mutate(child2,mutation_rate)
            new_population.extend([child1, child2])

        population = new_population

#### Intel® SigOpt Optimization

<p style='text-align: justify;'>    
Finally, the function below implements hyperparameter optimization using Intel® SigOpt
</p>

1. First, the function creates a connection to Intel® SigOpt using the provided API key. The connection is established with the Intel® SigOpt endpoint to send and receive data.

2. Next, an experiment is created in Intel® SigOpt. The experiment is defined with the name **Genetic Algorithm TSP Optimization** and the hyperparameter parameters to be optimized. Parameters include **population_size**, **max_generations**, **crossover_rate** and **mutation_rate**, each with its type and range of possible values.

3. Finally, a range of iterations must be defined where the Intel® SigOpt API will generate hyperparameters and test them using the **genetic_algorithm function**, obtaining as a result the best distance for the referred hyperparameters, at the end the best path to be performed will be returned, as well as the distance travelled.

In [1]:
# SigOpt Optimization Function
def sigopt_optimization():
    # Connect your API TOKEN
    conn = sigopt.Connection(client_token="YOUR API TOKEN")
    
    # Creating the sigOpt experiment and defining which hyperparameters will be optimized as well as the range covered
    experiment = conn.experiments().create(
        name="Genetic Algorithm TSP Optimization",
        parameters=[
            {"name": "population_size", "type": "int", "bounds": {"min": 20, "max": 100}},
            {"name": "max_generations", "type": "int", "bounds": {"min": 500, "max": 2000}},
            {"name": "crossover_rate", "type": "double", "bounds": {"min": 0.6, "max": 0.9}},
            {"name": "mutation_rate", "type": "double", "bounds": {"min": 0.1, "max": 0.4}}
        ],
        observation_budget = 50,  # Number of total iterations
    )
    
    # Optimization iterations
    for _ in range(experiment.observation_budget):
        # Suggested hyperparameters by sigOpt
        suggestion = conn.experiments(experiment.id).suggestions().create()
        hyperparameters = suggestion.assignments
        print(f"Running with hyperparameters: {hyperparameters}")

        # Run the genetic algorithm with suggested hyperparameters
        best_path = genetic_algorithm(
            hyperparameters["population_size"],
            hyperparameters["max_generations"],
            hyperparameters["crossover_rate"],
            hyperparameters["mutation_rate"],
        )

        # Calculate the total distance for the best path
        best_distance = total_distance(best_path)

        # Report the observation to SigOpt
        conn.experiments(experiment.id).observations().create(
            suggestion=suggestion.id, value=best_distance
        )

    return best_path

#### Run code

In [ ]:
if __name__ == "__main__":
    sigopt_optimization()

### ⊗ ACO

Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit, sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim ad minima veniam, quis nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur? Quis autem vel eum iure reprehenderit qui in ea voluptate velit esse quam nihil molestiae consequatur, vel illum qui dolorem eum fugiat quo voluptas nulla pariatur.

### ⊗ PSO

Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit, sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim ad minima veniam, quis nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur? Quis autem vel eum iure reprehenderit qui in ea voluptate velit esse quam nihil molestiae consequatur, vel illum qui dolorem eum fugiat quo voluptas nulla pariatur.

## Clear the memory

Before moving on, please execute the following cell to clear up the CPU memory. This is required to move on to the next notebook.

In [8]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

## Next

Congratulations, you have completed second part the learning objectives of the course! As a final exercise, successfully complete an applied problem in the assessment in [_06-bio-inspired-assessment.ipynb_](06-bio-inspired-assessment.ipynb).